Group members:

* Dalal Darwich
* Makram Noureddine
* Johny El Achkar

# Resource Constrained Project Scheduling Problem (RCPSP)
Consider a project that consists of `n` jobs. Jobs are performed using `m` types of resources (e.g. people, machines, etc.). The number of available units of resource `k` is denoted by `R[k]`. Job `i` has to be processed for `d[i]` time units (duration). During this time, `r[i, k]` units of each resource `k` are occupied. Additionally, precedence constraints are defined among jobs by table `p_tab` that contains the pairs `(i, j)`: for each `(i, j) in p_tab`, job `i` must be finished before job `j` is started. We say that `i` is a predecessor of `j`, or that `j` is a successor of `i`.

The goal of RCPSP is to find start and finish times of all jobs such that all jobs are completed, precedence and resource constraints are satisfied, and the project makespan (finish time of the last job) is minimized.

Problem instances can be obtained by calling `(file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)` where `idx` indicates the index of the instance.
* You should start your tests on instances with `idx in sel_instances`. 
* If you can, solve all instances with `idx in range(n_instances)`. 

In [1]:
import numpy as np
import pandas as pd
from gurobipy import *
data_dir = 'C:/Users/Makram/Downloads/data/Data/'

def read_rcpsp_data(idx):
    data_list = pd.read_csv(data_dir + 'all.txt', sep='\t')
    assert (idx >= 0) and (idx < data_list.shape[0]), "Index must be in the range"
    file = data_list.at[idx, 'File']
    with open(data_dir + file) as f:
        while True:
            data = f.readline().split()
            if data[0] == "jobs": break
        n = int(data[-1]) # number of jobs
        while True:
            data = f.readline().split()
            if data[-1] == "R": break
        m = int(data[-2]) # number of resources
        while True:
            data = f.readline().split()
            if data[0] == "PRECEDENCE": break
        f.readline() # skip line
        p_tab = [] # list of precedences
        for i in range(n):
            data = f.readline().split()
            assert int(data[0]) == i + 1, "First number should be the job number"
            assert int(data[1]) == 1, "Second number should be 1"
            assert int(data[2]) == len(data) - 3, "Third number should be the number of successors"
            for j in data[3:]: p_tab.append((i, int(j) - 1))
        for i in range(n - 1): assert len([(k, j) for (k, j) in p_tab if k == i]) > 0, "Each job except the last should have successors"
        assert len([(k, j) for (k, j) in p_tab if k == n - 1]) == 0, "Last job should have no successors"
        for i in range(1, n): assert len([(j, k) for (j, k) in p_tab if k == i]) > 0, "Each job except the first should have predecessors"
        assert len([(j, k) for (j, k) in p_tab if k == 0]) == 0, "First job should have no predecessors"
        while True:
            data = f.readline().split()
            if data[0] == "REQUESTS/DURATIONS:": break
        f.readline() # skip line
        f.readline() # skip line
        d = [0 for i in range(n)] # d[i] = duration of job i
        r = np.zeros((n, m), dtype=int) # r[i, j] = # of units of resource j needed for job i
        for i in range(n):
            data = f.readline().split()
            assert int(data[0]) == i + 1, "First number should be the job number"
            assert int(data[1]) == 1, "Second number should be 1"
            d[i] = int(data[2]) # job duration
            assert len(data) == m + 3, "The remaining numbers should be resource consumtion"
            r[i,:] = [int(j) for j in data[3:]]
        while True:
            data = f.readline().split()
            if data[0] == "RESOURCEAVAILABILITIES:": break
        f.readline() # skip line
        data = f.readline().split()
        R = [int(j) for j in data]
    return (file, n, m, d, p_tab, r, R)

n_instances = pd.read_csv(data_dir + 'all.txt', sep='\t').shape[0]
print('Number of all test instances:', n_instances)
sel_instances = [10 * i for i in range(n_instances // 10)]
print('Number of selected test instances:', len(sel_instances))
# print('Selected instances:')
# print('{:<4} {:20} {:>4} {:>4} {:>6} {:>10}'.format('idx', 'file', 'n', 'm', 'd_sum', 'p_tab_len'))
# for idx in sel_instances:
#     (file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)
#     print("{:<4} {:20} {:>4} {:>4} {:>6} {:>10}".format(idx, file, n, m, sum(d), len(p_tab)))
    
idx = 0
(file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)
print('\nData for example instance with idx =', idx)
print("file =", file)
print("n =", n)
print("m =", m)
print("d =", d)
print("p_tab =", p_tab)
print("R =", R)
print("r =", r)

Number of all test instances: 1560
Number of selected test instances: 156

Data for example instance with idx = 0
file = J30/j301_1.sm
n = 32
m = 4
d = [0, 8, 4, 6, 3, 8, 5, 9, 2, 7, 9, 2, 6, 3, 9, 10, 6, 5, 3, 7, 2, 7, 2, 3, 3, 7, 8, 3, 7, 2, 2, 0]
p_tab = [(0, 1), (0, 2), (0, 3), (1, 5), (1, 10), (1, 14), (2, 6), (2, 7), (2, 12), (3, 4), (3, 8), (3, 9), (4, 19), (5, 29), (6, 26), (7, 11), (7, 18), (7, 26), (8, 13), (9, 15), (9, 24), (10, 19), (10, 25), (11, 13), (12, 16), (12, 17), (13, 16), (14, 24), (15, 20), (15, 21), (16, 21), (17, 19), (17, 21), (18, 23), (18, 28), (19, 22), (19, 24), (20, 27), (21, 22), (22, 23), (23, 29), (24, 29), (25, 30), (26, 27), (27, 30), (28, 31), (29, 31), (30, 31)]
R = [12, 13, 4, 12]
r = [[ 0  0  0  0]
 [ 4  0  0  0]
 [10  0  0  0]
 [ 0  0  0  3]
 [ 3  0  0  0]
 [ 0  0  0  8]
 [ 4  0  0  0]
 [ 0  1  0  0]
 [ 6  0  0  0]
 [ 0  0  0  1]
 [ 0  5  0  0]
 [ 0  7  0  0]
 [ 4  0  0  0]
 [ 0  8  0  0]
 [ 3  0  0  0]
 [ 0  0  0  5]
 [ 0  0  0  8]
 [ 0  0  0  

# Model 1

The first model is based on the discrete time formulation DT where the goal will be to minimize the start time of the last job we have, which will be a dummy job with zero time and zero resources occupied

$T$: Sum of the durations of all jobs if they were performed each on its own and subsequently (Upper Bound)

Variables: $x_{it=1}$ if job $i$ is completed at time t, $0$ otherwise, for $i=0,\ldots,n-1$, for $t$ in T

Minimize:
    $\min \sum_{t=0}^{T} t.x_{n-1,t} + d_{n-1}$ for $t$ in T (The objective function in the mathematical formulation minimizes the makespan when reaching the last job.)

subject to:
    
$\sum_{t=0}^{T} x_{it}$ =1   for $i=0,\ldots,n-1$ (This contraint ensures that each job executes only once)

$\sum_{t=0}^{T} t.x_{jt}$ >= $\sum_{t=0}^{T}(t.x_{it}) + d_i$  for $i,j$ in $p_t$  (ensure the precedence relationships between jobs: job `i` must be finished before job `j` is started)

$\sum_{i=1}^{n-1} r_{ik}$ $\sum_{h=t-d_i +1}^{t} x_{ih} $ <= $R_k$  for $i=1,\ldots,n-1$, for $t$ in T, for $k$ in range $m$ (resource constraint)

$x_{it} ∈ {0,1}$   (specify the decision variables as binary.)


$\sum_{i=1}^{n-1} r_{ik}$ $\sum_{h=t-d_i +1}^{t} x_{ih} $ <= $R_k$  for $i=1,\ldots,n-1$, for $t$ in T, for $k$ in range $m$ 

In [2]:
def rcpsp(idx,UB,TimeLimit=60,LogToConsole=True):
    model=Model()
    T=UB
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit=TimeLimit
    arcs=[(i,t) for i in range(n) for t in range(T)]
    x=model.addVars(arcs,vtype=GRB.BINARY)
    obj=model.setObjective(quicksum(t*x[n-1,t]+d[n-1] for t in range(T)),GRB.MINIMIZE)
    model.addConstrs(quicksum(x[i,t] for t in range(T))== 1 for i in range(n))
    model.addConstrs((quicksum(t*x[j,t] for t in range(T))>= (quicksum(t*x[i,t] for t in range(T))+d[i])) for i,j in p_tab)
    model.addConstrs(quicksum(r[i,k]*x[i,h] for i in range(1,n-1) for h in range(max(0,t-d[i]+1),t)) <= R[k] for k in range(m) for t in range(T))
    model.optimize()

    if model.SolCount > 0:
        solution=[(i,t) for (i,t) in arcs if x[i,t].X>0.5]
    else:
        solution=[]

    return(solution, model.ObjVal,model.ObjBound,model.NumVars, model.NumConstrs,model.MIPGap,model.runTime)  

In [39]:
for idx in sel_instances:
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars, NumConstrs, MIPGap,runTime) = rcpsp(idx,sum(d), 60, False)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

IDX: 0 File:  J30/j301_1.sm ObjVal:  42.0 ObjBound:  42.0 Num of Variables:  5056 Num of Constrs: 712 MIPGap: 0.0% runtime: 0.3667888641357422s
IDX: 10 File:  J30/j302_1.sm ObjVal:  35.0 ObjBound:  35.0 Num of Variables:  4768 Num of Constrs: 676 MIPGap: 0.0% runtime: 0.2509136199951172s
IDX: 20 File:  J30/j303_1.sm ObjVal:  72.0 ObjBound:  72.0 Num of Variables:  6560 Num of Constrs: 900 MIPGap: 0.0% runtime: 0.30584144592285156s
IDX: 30 File:  J30/j304_1.sm ObjVal:  49.0 ObjBound:  49.0 Num of Variables:  4544 Num of Constrs: 648 MIPGap: 0.0% runtime: 0.09794425964355469s
IDX: 40 File:  J30/j305_1.sm ObjVal:  48.0 ObjBound:  48.0 Num of Variables:  5152 Num of Constrs: 724 MIPGap: 0.0% runtime: 1.7896652221679688s
IDX: 50 File:  J30/j306_1.sm ObjVal:  56.0 ObjBound:  56.0 Num of Variables:  5696 Num of Constrs: 792 MIPGap: 0.0% runtime: 1.8357524871826172s
IDX: 60 File:  J30/j307_1.sm ObjVal:  55.0 ObjBound:  55.0 Num of Variables:  5344 Num of Constrs: 748 MIPGap: 0.0% runtime: 0.31

Looking at the above results, we notice that the deployed model manged to solve many of the test instances into optimality. Some of the test instances weren't solved completely to optimality and recorded a MIPGap percentage rangin between 0% and 100%. 

On the other side, we noticed that some of the instances didn't solve at all giving us an OBJVal and MIPGap equal to infinity. These instances might require a higher timelimit to be solved to optimality.

# Model 2:

The second model is basically based on the first model we implemented but with different approcahes for the timelimit and upper bounds used.

To moderate this characteristic, we perform a preprocessing phase aiming at reducing activity time windows by standard precedence and resource constraint propagation. Such preprocessing
allows a high reduction of the number of binary variables and strengthen the relaxation of formulations DT.

The timelimit of 60 seconds in the second model will be divided into two parts: 

1- In the first part, the intial model will run for 20% of the assigned timeframe or 12 seconds and solve all the test instances again. If a test instance is solved to optimality under 12 seconds, the results will be displayed and the model will go to the next instance. In case the test instance wasn't solved into optomilaty within 12 seconds, we will proceed to the second part.


2- In the second part, the test instance that didn't converge under the 12 seconds timeframe will be re-evaluated using the remaining 48 seconds of the initial 60 seconds we set,  with a new upper bound set.

The new upper bound will get will depends on two different scenarios:

   a- If the MIPGap result we got during the first part is greater than 0% and less than 3%, the upper bound used will still be equal to the sum of durations assigned to each job. Using this approach, all jobs with such low MIPGap will converge under the span of 48 seconds and the model will solve to optimality.
    
   b- If the MIPGap result we got during the first part is greater than 3%, we will have two different scenarios for the upperbound to be used:
    
* If the initial objective value is equal to infinity, the upper bound used will be equal to the sum of durations assigned to each job.
        
* If the initial objective value is equal to a certain interger number, the new upper bound to be used will be equal to the objective value number we got for this job in the first part we run.

After the new upper bound is set based on the MIPGap value we got, the model will re-run agian and try to reach optimality. However, even using this method won't guarrentee that all test instances will be solved to optimality as some jobs requires longer timelimit. This will depends on the computational power of the machine we are using.

However, we will run the second model for some particular instances we choose but with a higher timelimit and see if the model will reach optimality or not.

In [2]:
def rcpsp1(idx,UB,TimeLimit=60,LogToConsole=True):
    model=Model()
    #UB=sum(d)
    T=UB
    print("Upper Bound:", UB)
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit=TimeLimit
    arcs=[(i,t) for i in range(n) for t in range(T)]
    x=model.addVars(arcs,vtype=GRB.BINARY)
    obj=model.setObjective(quicksum(t*x[n-1,t]+d[n-1] for t in range(T)),GRB.MINIMIZE)
    model.addConstrs(quicksum(x[i,t] for t in range(T))== 1 for i in range(n))
    model.addConstrs((quicksum(t*x[j,t] for t in range(T))>= (quicksum(t*x[i,t] for t in range(T))+d[i])) for i,j in p_tab)
    model.addConstrs(quicksum(r[i,k]*x[i,h] for i in range(1,n-1) for h in range(max(0,t-d[i]+1),t)) <= R[k] for k in range(m) for t in range(T))
    model.optimize()

    if model.SolCount > 0:
        solution=[(i,t) for (i,t) in arcs if x[i,t].X>0.5]
    else:
        solution=[]

    return(solution, model.ObjVal,model.ObjBound,model.NumVars, model.NumConstrs,model.MIPGap,model.runTime)  

In [55]:
import math

for idx in sel_instances:
    TimeLimit=60
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,sum(d),TimeLimit*0.2,False)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

    if (MIPGap > 0.000001) & (MIPGap < 0.03):
        
        print("Improved Solution with ObjVal set as new upper bound:")
        T=sum(d)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Va  riables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))
    
    elif MIPGap > 0.03:
        
        print("Improved Solution with ObjVal set as new upper bound:")
        if ObjVal == math.inf:
            T=sum(d)
        else:
            T=int(ObjVal+1)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))


Upper Bound: 158
IDX: 0 File:  J30/j301_1.sm ObjVal:  42.0 ObjBound:  42.0 Num of Variables:  5056 Num of Constrs: 712 MIPGap: 0.0% runtime: 0.3547954559326172s
Upper Bound: 149
IDX: 10 File:  J30/j302_1.sm ObjVal:  35.0 ObjBound:  35.0 Num of Variables:  4768 Num of Constrs: 676 MIPGap: 0.0% runtime: 0.28082275390625s
Upper Bound: 205
IDX: 20 File:  J30/j303_1.sm ObjVal:  72.0 ObjBound:  72.0 Num of Variables:  6560 Num of Constrs: 900 MIPGap: 0.0% runtime: 0.28782081604003906s
Upper Bound: 142
IDX: 30 File:  J30/j304_1.sm ObjVal:  49.0 ObjBound:  49.0 Num of Variables:  4544 Num of Constrs: 648 MIPGap: 0.0% runtime: 0.08794975280761719s
Upper Bound: 161
IDX: 40 File:  J30/j305_1.sm ObjVal:  48.0 ObjBound:  48.0 Num of Variables:  5152 Num of Constrs: 724 MIPGap: 0.0% runtime: 1.74298095703125s
Upper Bound: 178
IDX: 50 File:  J30/j306_1.sm ObjVal:  56.0 ObjBound:  56.0 Num of Variables:  5696 Num of Constrs: 792 MIPGap: 0.0% runtime: 1.544107437133789s
Upper Bound: 167
IDX: 60 File:  

## Model 2 Selected test instances:

In this part, we will run the second model for some particular instances that didn't converge at all in the above part but with a higher timelimit of 240 seconds and see if the model will reach optimality or not.

As a start, we will take the instances 1110,1120,1130 and 1140. As we can see in the previous results, the model didn't return any value for the objective value. The value of ObjVal returned was equal to infinity and so the MPIGap. 

In [56]:
import math

for idx in sel_instances[111:115]:
    TimeLimit=240
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,sum(d),TimeLimit*0.2,False)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

    if (MIPGap > 0.000001) & (MIPGap < 0.03):
        print("Improved Solution with ObjVal set as new upper bound:")
        T=sum(d)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))
    elif MIPGap > 0.03:
        print("Improved Solution with ObjVal set as new upper bound:")
        if ObjVal == math.inf:
            T=sum(d)
        else:
            T=int(ObjVal+1)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))


Upper Bound: 646
IDX: 1110 File:  J120/j12016_1.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  78812 Num of Constrs: 2889 MIPGap: inf% runtime: 48.02130126953125s
Improved Solution with ObjVal set as new upper bound:
Upper Bound: 646
IDX: 1110 File:  J120/j12016_1.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  78812 Num of Constrs: 2889 MIPGap: inf% runtime: 192.01823043823242s
Upper Bound: 710
IDX: 1120 File:  J120/j12017_1.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  86620 Num of Constrs: 3145 MIPGap: inf% runtime: 48.0192985534668s
Improved Solution with ObjVal set as new upper bound:
Upper Bound: 710
IDX: 1120 File:  J120/j12017_1.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  86620 Num of Constrs: 3145 MIPGap: inf% runtime: 192.01922798156738s
Upper Bound: 716
IDX: 1130 File:  J120/j12018_1.sm ObjVal:  inf ObjBound:  101.0 Num of Variables:  87352 Num of Constrs: 3169 MIPGap: inf% runtime: 48.427072525024414s
Improved Solution with ObjVal set as new upper bound:
Uppe

### Results Discussion

As we can in the above results, for instance 1110 & 1120, even the 240 seconds weren't enough to provide us with any sort of ObjVal. The value returned was still eual to infinity.

For instance 1130, we notice that during the first phase of 48 seconds, the model didn't return any optimal ObjVal. Whereas, in the second phase of the model and after running for 192 seconds instead of the 48 seconds, we get an initial value of ObjVal equal to 712. However, the MIPGap is still very high and far away from optimality. We might even consider later increasing the timelimit to 600 seconds and see if we will get any optimal results.

As for the last tested instance 1140, we notice that the model reached optimality in around 27.5 seconds, same time it took us to reach optimality in the previous results.

### Test Instance #1130

In the following model we will increase the timelimit to 600 seconds and comapre which model did best. Model 1 will use the 600sec to solve for optimality whereas using Model 2 the timelimit is divided into 120 seconds for phase 1 and 480 seconds for phase 2 to check if the model will return any optimal solution. However it's to be noted that increasing the timelimit to such high values wouldn't be recommended if we decide to use the model we built for real operation as in reality, we might have a high number of  files with very high number of jobs that needs to planned for. So deploying a model that will take too much time to solve weon't be efficient and recommened as perhaps we need to do the planning on daily hourly or even daily basis.

#### Model 1

In [3]:
for idx in sel_instances[113:114]:
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars, NumConstrs, MIPGap,runTime) = rcpsp(idx,sum(d), 600, True)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

Academic license - for non-commercial use only - expires 2022-08-30
Using license file C:\Users\Makram\gurobi.lic
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3169 rows, 87352 columns and 2046834 nonzeros
Model fingerprint: 0xb3ff0f90
Variable types: 0 continuous, 87352 integer (87352 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 33 rows and 4407 columns (presolve time = 6s) ...
Presolve added 86 rows and 0 columns
Presolve removed 0 rows and 4292 columns
Presolve time: 6.04s
Presolved: 3255 rows, 83060 columns, 1807351 nonzeros
Variable types: 0 continuous, 83060 integer (82941 b

#### Model 2

In [10]:
import math

for idx in sel_instances[113:114]:
    TimeLimit=600
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,sum(d),TimeLimit*0.2,True)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

    if (MIPGap > 0.000001) & (MIPGap < 0.03):
        print("Improved Solution with ObjVal set as new upper bound:")
        T=sum(d)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,True)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))
    elif MIPGap > 0.03:
        print("Improved Solution with ObjVal set as new upper bound:")
        if ObjVal == math.inf:
            T=sum(d)
        else:
            T=int(ObjVal+1)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,True)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))


Upper Bound: 716
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 120.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3169 rows, 87352 columns and 2046834 nonzeros
Model fingerprint: 0xb3ff0f90
Variable types: 0 continuous, 87352 integer (87352 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 33 rows and 4407 columns (presolve time = 5s) ...
Presolve added 86 rows and 0 columns
Presolve removed 0 rows and 4292 columns
Presolve time: 6.55s
Presolved: 3255 rows, 83060 columns, 1807351 nonzeros
Variable types: 0 continuous, 83060 integer (82941 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    

Looking at the above results, we notice that model 1 performed better by reducing the GAp till 17% whereas for model 2 the MIPGap was reduced to 22.6%.

However it is to be noted that the reduction in gap from 85% to 17% was at the cost of increasing the timelimit by 10 times which is not something efficient to be deploy taking into consideration the large number of unsolved test instances we got. 

### Looping through the range of n_instances

In the below model, we will deploy the second model we developped and test it on all the instances we have using a timelimit of 60 seconds. The total number of  instances we had from the beginning was equal to 1560, so we will check how efficient our model is and how much time it might take to solve all the instances.

#NB: The run time for all instances took more than 20 Hours

In [ ]:
for idx in range(n_instances):
    TimeLimit=60
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,sum(d),TimeLimit*0.2,False)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

    if (MIPGap > 0.000001) & (MIPGap < 0.03):
        
        print("Improved Solution with ObjVal set as new upper bound:")
        T=sum(d)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))
    
    elif MIPGap > 0.03:
        
        print("Improved Solution with ObjVal set as new upper bound:")
        if ObjVal == math.inf:
            T=sum(d)
        else:
            T=int(ObjVal+1)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))


Upper Bound: 158
IDX: 0 File:  J30/j301_1.sm ObjVal:  42.0 ObjBound:  42.0 Num of Variables:  5056 Num of Constrs: 712 MIPGap: 0.0% runtime: 0.3208122253417969s
Upper Bound: 160
IDX: 1 File:  J30/j301_2.sm ObjVal:  45.0 ObjBound:  45.0 Num of Variables:  5120 Num of Constrs: 720 MIPGap: 0.0% runtime: 0.3877735137939453s
Upper Bound: 141
IDX: 2 File:  J30/j301_3.sm ObjVal:  46.0 ObjBound:  46.0 Num of Variables:  4512 Num of Constrs: 644 MIPGap: 0.0% runtime: 0.23386573791503906s
Upper Bound: 184
IDX: 3 File:  J30/j301_4.sm ObjVal:  59.0 ObjBound:  59.0 Num of Variables:  5888 Num of Constrs: 816 MIPGap: 0.0% runtime: 0.4447307586669922s
Upper Bound: 119
IDX: 4 File:  J30/j301_5.sm ObjVal:  35.0 ObjBound:  35.0 Num of Variables:  3808 Num of Constrs: 556 MIPGap: 0.0% runtime: 0.4757251739501953s
Upper Bound: 123
IDX: 5 File:  J30/j301_6.sm ObjVal:  45.0 ObjBound:  45.0 Num of Variables:  3936 Num of Constrs: 572 MIPGap: 0.0% runtime: 0.2688312530517578s
Upper Bound: 148
IDX: 6 File:  J3

###### The computer shutted down for updates, so we had to continue the rest of the instances seperately

In [3]:
for idx in range(n_instances)[1414:]:
    TimeLimit=60
    (file, n, m, d, p_tab, r, R)= read_rcpsp_data(idx)
    (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,sum(d),TimeLimit*0.2,False)
    print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))

    if (MIPGap > 0.000001) & (MIPGap < 0.03):
        
        print("Improved Solution with ObjVal set as new upper bound:")
        T=sum(d)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))
    
    elif MIPGap > 0.03:
        
        print("Improved Solution with ObjVal set as new upper bound:")
        if ObjVal == math.inf:
            T=sum(d)
        else:
            T=int(ObjVal+1)
        (solution,ObjVal,ObjBound,NumVars,NumConstrs,MIPGap,runTime) = rcpsp1(idx,T,TimeLimit*0.8,False)
        print("IDX:", idx, "File: ",file,"ObjVal: ", ObjVal,"ObjBound: ", ObjBound,"Num of Variables: ", NumVars,"Num of Constrs:",NumConstrs,"MIPGap: {}%".format(MIPGap*100),"runtime: {}s".format(runTime))


Academic license - for non-commercial use only - expires 2022-08-30
Using license file C:\Users\Makram\gurobi.lic
Upper Bound: 657
IDX: 1414 File:  J120/j12046_5.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  80154 Num of Constrs: 3007 MIPGap: inf% runtime: 12.030071258544922s
Improved Solution with ObjVal set as new upper bound:
Upper Bound: 657
IDX: 1414 File:  J120/j12046_5.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  80154 Num of Constrs: 3007 MIPGap: inf% runtime: 48.05082130432129s
Upper Bound: 599
IDX: 1415 File:  J120/j12046_6.sm ObjVal:  inf ObjBound:  97.0 Num of Variables:  73078 Num of Constrs: 2775 MIPGap: inf% runtime: 12.021076202392578s
Improved Solution with ObjVal set as new upper bound:
Upper Bound: 599
IDX: 1415 File:  J120/j12046_6.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  73078 Num of Constrs: 2775 MIPGap: inf% runtime: 48.04035186767578s
Upper Bound: 641
IDX: 1416 File:  J120/j12046_7.sm ObjVal:  inf ObjBound:  0.0 Num of Variables:  78202 Num of 

### n_instances results dicsussion and recommendation about the deployed models

The above results shows us that all many of the instances were solved to optimality whereas for many other instances where we had large actitivity number that needs to be planned for, the model deployed didn't help at all in providing any sort of optimal results.

In terms of computational time, we would like to highlight that the  model 2 took almost 24 hours to solve all the instances we had using a timelimit of just 60 seconds. Since, many of the intances didn't return any results or even solved to optimality, we can say that the deployed model might not be the optimal model we shoul deploy in real life operations. 

Also, increasing the timelimit to more than 60 seconds might not even be the perfect solution as this will yield longer time computation, the thing that we might not have in actual operations as we might be talking about daily operations that we need to plan for. 

The models might be useful for the small file cards where the number of activities is relatively small such as 32 and where the total timespan for planning isn't that big.

Model 2 performed better in reducing the MIPGap for the files with small number of activities whereas Model 1 performed better to an extent for the large files, especially when the phase 1 of Model 2 didn't converge to any new OBJVal that will help us in reducin gthe upper bound for the second phase.

It is recommeded to check for other more advanced type of models we can deploy to solve the RCPSP problem and comapre the results we obtain in terms of MIPGap mimimaztion and the overall computation time. Such models can be as follow:
    
* Flow-based continuous-time formulation which is based on 
* Start/End Event-based formulation
* On/Off Event-based formulation
  
